In [ ]:
import pandas as pd
import os
import pickle
import numpy as np
import json

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"

In [ ]:

df_c3 = pd.read_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset.csv"))
# df_c3['file_path'] = df_c3['file_path'].str.replace('Groups/FAMLI/Restricted_access_data/Ultrasound/', '').str.replace('Groups/FAMLI/Deidentified_data/Ultrasound/', '').str.replace(f'Dataset_C3', f'Dataset_C3_masked_resampled_256_spc075').str.replace('.mp4', '.nrrd').str.replace('.dcm', '.nrrd')
# df_c3_res = df_c3[df_c3['file_path'].apply(lambda x: os.path.exists(os.path.join(mount_point, str(x))))]
# df_c3_res = df_c3_res.query("cine == 1")
# df_c3_res.to_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_masked_resampled_256_spc075.csv"), index=False)

In [ ]:
# df_c3.query('cine == 0 and tag == "AC"').sample(1).iloc[0]['file_path']

In [ ]:
df_c3_res = pd.read_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_masked_resampled_256_spc075.csv"))




In [ ]:
df_c3_res['tag'].value_counts().to_dict()

In [ ]:
# df_c3_res_flyto = df_c3_res[df_c3_res['tag'].isin(['AC', 'FL', 'BPD', 'HC', 'TCD', 'HL', 'CRL'])]
# df_c3_res_flyto.to_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_masked_resampled_256_spc075_flyto.csv"), index=False)

In [ ]:
# df_c3_res_flyto = pd.read_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_masked_resampled_256_spc075_flyto.csv"))

In [ ]:
df_c3_frames = pd.read_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075.csv"))
df_c3_frames["file_path_orig"] = df_c3_frames["file_path"].apply(lambda x: x.replace("extract_frames_blind_sweeps/", '').replace("/" + os.path.basename(x), '') + '.nrrd')
df_c3_frames = df_c3_frames.merge(df_c3_res.rename(columns={"file_path": "file_path_orig"}), on='file_path_orig', how='left')

df_c3_frames = df_c3_frames[~df_c3_frames['tag'].isna()]
df_c3_frames.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged.csv"), index=False)

In [ ]:
len(df_c3_frames)

In [ ]:
df_annot = pd.read_csv(os.path.join(mount_point, "CSV_files/c3_blindsweep_annotation_labels_merged.csv"))
df_annot_file_path_update = []

for idx, row in df_annot.iterrows():
    df_annot_file_path_update.append('extract_frames_blind_sweeps/' + os.path.splitext(row['file_path'])[0] + "/" + str(row['frame_index']) + '.nrrd')
df_annot['file_path'] = df_annot_file_path_update
df_annot = df_annot[df_annot['file_path'].apply(lambda x: os.path.exists(os.path.join(mount_point, x)))]


In [ ]:
df_annot.columns

In [ ]:
df_annot['annotation_label'].value_counts().to_dict()

In [ ]:
df_c3_frames['tag'].value_counts()

In [ ]:
# df_c3_frames_ds = []

# df_c3_frames_ds.append(df_c3_frames.query("tag == 'AC'"))
# df_c3_frames_ds.append(df_c3_frames.query("tag != 'AC'").sample(n=30000, random_state=42))
# df_c3_frames_ds = pd.concat(df_c3_frames_ds)
# df_c3_frames_ds.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_nonac.csv"), index=False)

In [ ]:
df_c3_frames_ds = pd.read_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_nonac.csv"))
df_c3_frames_ds['tag'].value_counts().to_dict()

In [ ]:
# df_c3_frames_ds_ac = df_c3_frames_ds[df_c3_frames_ds['tag'] == 'AC']
# df_c3_frames_ds_ac.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_only.csv"), index=False)
df_c3_frames_ds_ac = pd.read_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_only.csv"))
df_c3_frames_ds_ac['annotation_label'] = 'high_measurable'

In [ ]:
df_c3_frames_ds_v2 = []

df_c3_frames_ds_v2.append(df_c3_frames_ds_ac[['file_path', 'pid', 'annotation_label']])


non_ac = df_c3_frames.query("tag != 'AC'").sample(n=30000, random_state=42)
non_ac['annotation_label'] = 'reject'

df_c3_frames_ds_v2.append(non_ac[['file_path', 'pid', 'annotation_label']])
df_c3_frames_ds_v2.append(df_annot[['file_path', 'pid', 'annotation_label']])
df_c3_frames_ds_v2 = pd.concat(df_c3_frames_ds_v2).drop_duplicates().reset_index(drop=True)
df_c3_frames_ds_v2.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_final_multi_label_ds.csv"), index=False)

In [ ]:
df_c3_frames_ds_v2['annotation_label'].value_counts().to_dict()

In [ ]:
class_dict = {'reject': 0,
 'low_visible': 0,
 'high_visible': 1,
 'low_measurable': 1,
 'high_measurable': 1,}
json.dump(class_dict, open(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_final_multi_label_ds_class_dict.json"), 'w'))

class_dict_multi = {'reject': 0,
 'low_visible': 1,
 'high_visible': 1,
 'low_measurable': 2,
 'high_measurable': 2,}
json.dump(class_dict_multi, open(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_final_multi_label_ds_class_dict_multi.json"), 'w'))

class_dict_multi = {'reject': [0.75, 0.25, 0.0, 0.0],
 'low_visible': [0.25, 0.5, 0.25, 0.0],
 'high_visible': [0.0, 0.25, 0.5, 0.25],
 'low_measurable': [0.0, 0.0, 0.7, 0.25],
 'high_measurable': [0.0, 0.0, 0.0, 1.0],}
json.dump(class_dict_multi, open(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_final_multi_label_ds_class_dict_multi_soft.json"), 'w'))

In [ ]:
score_dict = {'reject': 0,
 'low_visible': 0,
 'high_visible': 0.5,
 'low_measurable': 0.75,
 'high_measurable': 1}
json.dump(score_dict, open(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_final_multi_label_ds_score_dict.json"), 'w'))

In [ ]:
df_c3_frames_ds.columns

In [ ]:
class_dict = {'AC': 1,
 'FL': 0,
 'BPD': 0,
 'TCD': 0,
 'HL': 0,
 'CRL': 0,
 'C3': 0,
 'C2': 0,
 'L0': 0,
 'R0': 0,
 'C1': 0,
 'L1': 0,
 'M': 0,
 'R1': 0,
 'C4': 0,
 'HC': 0}

json.dump(class_dict, open(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_nonac_classdict.json"), 'w'))

In [ ]:
df_c3_frames_ds

In [ ]:
df_merged = pd.read_csv(os.path.join(mount_point, 'CSV_files/c3_blindsweep_annotation_labels_merged.csv'))

annot_pids = df_merged['pid'].unique()

df_c3_frames_ds_train = df_c3_frames_ds[~df_c3_frames_ds['pid'].isin(annot_pids)]
df_c3_frames_ds_test = df_c3_frames_ds[df_c3_frames_ds['pid'].isin(annot_pids)]

# df_c3_frames_ds_train.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_nonac_train.csv"), index=False)
# df_c3_frames_ds_test.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_balanced_ac_nonac_test.csv"), index=False)


In [ ]:
# df_c3_blindsweep = df_c3[df_c3['tag'].isin(['M', 'L0', 'L1', 'R0', 'R1', 'C1', 'C2', 'C3', 'C4', ])]
# df_c3_blindsweep['file_path'] = df_c3_blindsweep['file_path'].str.replace('Groups/FAMLI/Restricted_access_data/Ultrasound/', '').str.replace('Groups/FAMLI/Deidentified_data/Ultrasound/', '').str.replace(f'Dataset_C3', f'Dataset_C3_masked_resampled_256_spc075').str.replace('.mp4', '.nrrd').str.replace('.dcm', '.nrrd')

# df_c3_blindsweep = df_c3_blindsweep[df_c3_blindsweep['file_path'].apply(lambda x: os.path.exists(os.path.join(mount_point, str(x))))]
# df_c3_blindsweep.to_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_blind_sweeps_only.csv"), index=False)
# df_c3_blindsweep.sample(n=1000).to_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_blind_sweeps_only_sample1000.csv"), index=False)

In [ ]:
df_c3_blindsweep = pd.read_csv(os.path.join(mount_point, "CSV_files/c3_instance_analysis_dataset_blind_sweeps_only.csv"))

In [ ]:
df_c3_frames

In [ ]:
df_annot['file_path_orig'] = df_annot['file_path'].apply(lambda x: x.replace('extract_frames_blind_sweeps/', '').replace('/' + os.path.basename(x), '') + '.nrrd')

In [ ]:
df_c3_frames_orig = df_c3_frames[['file_path', 'file_path_orig']]
df_c3_frames_orig['frame_index'] = df_c3_frames_orig['file_path'].apply(lambda x: int(os.path.splitext(os.path.basename(x))[0]))


In [ ]:
df_c3_frames_orig

In [ ]:
df_annot_orig = df_annot[['file_path_orig', 'file_path', 'frame_index', 'annotation_label']]

In [ ]:
df_annot_orig

In [ ]:
df_c3_frames_orig

In [ ]:
df_annot_merged_full = df_c3_frames_orig[df_c3_frames_orig['file_path_orig'].isin(df_annot_orig['file_path_orig'].drop_duplicates())].merge(
    df_annot_orig[['file_path_orig', 'frame_index', 'annotation_label']],
    on=['file_path_orig', 'frame_index'],
    how='left'
)
df_annot_merged_full['annotation_label'] = df_annot_merged_full['annotation_label'].fillna('reject')


df_annot_merged_full['annotation_label'].value_counts().to_dict()

In [ ]:
df_annot_merged_full.columns

In [ ]:
df_annot_merged_full = df_annot_merged_full.merge(df_c3_res.rename(columns={"file_path": "file_path_orig"})[['file_path_orig', 'pid', 'tag', 'visitid']],
    on='file_path_orig',
    how='left'
)

df_annot_merged_full = pd.concat([df_c3_frames_ds_ac[['file_path_orig', 'file_path', 'pid', 'annotation_label']], df_annot_merged_full], ignore_index=True).drop_duplicates().reset_index(drop=True)

df_annot_merged_full.to_csv(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_full_annotations.csv"), index=False)

In [ ]:
df_annot_merged_full['annotation_label'].value_counts().to_dict()

In [ ]:
full_annot_class_dict = {'reject': 0,
 'low_visible': 1, 
 'high_visible': 2,
 'low_measurable': 3,
 'high_measurable': 4,}

json.dump(full_annot_class_dict, open(os.path.join(mount_point, "CSV_files/extract_frames_blind_sweeps_Dataset_C3_masked_resampled_256_spc075_merged_full_annotations_class_dict.json"), 'w'))